# packages and imports

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import mean

from data_makers import *
import utils
from utils import *
import mean_model
from mean_model import meanModel
import ML_models
from ML_models import *

from Nikhil_code import data_preprocessing as dp_nb
from Nikhil_code import model_selection as ms_nb
from Nikhil_code import testing as t_nb
import Nikhil_code.Data_imports as di_nb
import Nikhil_code.pairs_train_test_split as tts_nb
import Nikhil_code.Learning_curve as lc_nb

import sklearn
from sklearn.inspection import permutation_importance

from sklearn.metrics import mean_squared_error, r2_score


2023-10-06 12:30:42.267617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 12:30:42.430545: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-06 12:30:42.437075: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-06 12:30:42.437094: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Feature selection (FS) and data creating for each drug

In [3]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data(transformation='norm')
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

/data/home/bt22941/ML_notebook/Nikhil_code/Data_imports.py:46: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  uniprot_ids = pd.read_csv(


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10


/data/home/bt22941/ML_notebook/Nikhil_code/Data_imports.py:81: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  cell_names_raw = pd.read_csv(f'{gdsc_path}/downloaded_data/gdsc_cell_names.csv', skiprows=1, skipfooter=1)


num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

In [4]:
## PHOSPHO
landmark_genes = pd.read_csv(
    f'data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

X_L1000 = landmark_X_maker(phospho_ls, landmark_genes['Symbol'])

#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    X_L1000, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line pairs
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

(11583, 2056)
(11583, 2401)


In [5]:
##PROTEOMIC
#use the same landmark genes for fs with prot data
landmark_genes = pd.read_csv(
    f'data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

#find overlapping landmark genes and prot features
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), prot.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_main_prot = dp_nb.create_all_drugs(
    prot[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_main_prot.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)
x_all_prot.shape
X_main_prot = X_main_maker(x_all_prot, x_drug, short = False)
print(X_main_prot.shape)

(11583, 1066)


In [6]:
#one hot data creation for all drugs
x_hot, x_drug_hot, y_main_onehot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot
x_hot.columns = np.arange(len(x_drug.columns), len(x_drug.columns) + len(x_hot.columns))
print(x_hot.shape)
X_main_onehot = X_main_maker(x_hot, x_drug, short = False)
print(X_main_onehot.shape)

(11583, 38)
(11583, 383)


In [7]:
#train-test-split
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 88)
print(f'dd_X_main.shape: {X_main_phos.shape}')
print(f'X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}')

dd_X_main.shape: (11583, 2401)
X_train.shape: (9213, 2401), X_test.shape: (2370, 2401)


# Feature Selection

In [ ]:
#read in the RANDOM FOREST feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
#read in the XGBOOST feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/xgb_X_main/xgb_feat_select_rs0.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
#create a new X dataframe with the selected features
X_features = phospho_ls.reindex(feature_list[:2056],axis="columns")

In [ ]:
## PHOSPHO
#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    X_features, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

# Early stopping and parameters

In [8]:
# set early stopping and learning rate scheduler
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler)
callbacks_list =  [es,lr_scheduler]

In [9]:
# set parameters and train
num_epochs = 200 # max epoch for training
learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 42

# General metrics- Decision trees

In [13]:
print(f'X_main_phos shape: {X_main_phos.shape}')
print(f'X_main_onehot shape: {X_main_onehot.shape}')
print(f'y_main_phos shape: {y_main_phos.shape}')

X_main_phos shape: (11583, 2401)
X_main_onehot shape: (11583, 383)
y_main_phos shape: (11583,)


phospho

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
 
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_standard_error = []
model_MSE = []
model_pearson = []
model_type = 'rfr'

for seed in rs:
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    print(f'Mixed-Set Model')
    #MIXED SET
    #MODEL
    if model_type == 'rfr':
        r2, MSE, pearson, standard_error = rfr(X_main_onehot, y_main_phos, test_size=0.2, random_state=seed, iterations=3)
    elif model_type == 'xgb':
        r2, MSE, pearson, standard_error = xgb(X_main_onehot, y_main_phos, test_size=0.2, random_state=seed, iterations=3)
    mixed_set_r2.append(r2)

    print(f'Model: {model_type}')
    #MODEL
    if model_type == 'rfr':
        r2, MSE, pearson, standard_error = rfr(X_main_phos, y_main_phos, test_size=0.2, random_state=seed, iterations=3)
    elif model_type == 'xgb':
        r2, MSE, pearson, standard_error = xgb(X_main_phos, y_main_phos, test_size=0.2, random_state=seed, iterations=3)
    model_r2.append(r2)
    model_standard_error.append(standard_error)
    model_MSE.append(MSE)
    model_pearson.append(pearson)

In [ ]:
table_make('rfr_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_standard_error, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_standard_error', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/NormalisedX_rfr_metrics.csv')
           

proteomic

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_standard_error = []
model_MSE = []
model_pearson = []
model_type = 'xgb'

for seed in rs:
    print(f'Seed: {seed}')
    
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_prot, y=y_main_prot, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    print(f'Mixed-Set Model')
    #MIXED SET
    #MODEL
    if model_type == 'rfr':
        r2, MSE, pearson, standard_error = rfr(X_main_onehot, y_main_prot, test_size=0.2, random_state=seed, iterations=3)
    elif model_type == 'xgb':
        r2, MSE, pearson, standard_error = xgb(X_main_onehot, y_main_prot, test_size=0.2, random_state=seed, iterations=3)
    mixed_set_r2.append(r2)

    #MODEL
    if model_type == 'rfr':
        r2, MSE, pearson, standard_error = rfr(X_main_prot, y_main_prot, test_size=0.2, random_state=seed, iterations=3)
    elif model_type == 'xgb':
        r2, MSE, pearson, standard_error = xgb(X_main_prot, y_main_prot, test_size=0.2, random_state=seed, iterations=3)
    model_r2.append(r2)
    model_standard_error.append(standard_error)
    model_MSE.append(MSE)
    model_pearson.append(pearson)

In [ ]:
table_make('rfr_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_standard_error, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_standard_error', 'model_MSE', 'model_pearson'], 
           file = 'plots/proteomic/NormalisedX_xgb_metrics.csv')
           

# General Metric- Late integration

In [11]:
print(f'x_all_phos shape: {x_all_phos.shape} y_main_phos shape: {y_main_phos.shape}')
print(f'x_drug shape: {x_drug.shape} y_main_phos shape: {y_main_phos.shape}')
print(f'X_main_onehot shape: {X_main_onehot.shape} y_main_onehot shape: {y_main_phos.shape}')

x_all_phos shape: (11583, 2056) y_main_phos shape: (11583,)
x_drug shape: (11583, 345) y_main_phos shape: (11583,)
X_main_onehot shape: (11583, 383) y_main_onehot shape: (11583,)


In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_MSE = []
model_pearson = []

for seed in rs:
    
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    
    
    
    print(f'Mixed-Set Model')
    #MIXED SET
    #phospho
    xo_train, xo_test = cell_line_split(x_hot, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_phos, y=None, test_size=0.2, random_state = seed)
    
    ## Code for model building and training

    model = build_CNN(xo_train, xd_train, learning_rate, momentum, initializer_seed, mtype = 'regression')
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
        
    mixed_set_r2.append(r2)

    
    
    
    print(f'Model: CNN')
    
    #phospho
    xo_train, xo_test = cell_line_split(x_all_phos, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_phos, y=None, test_size=0.2, random_state = seed)
    

    ## Code for model building and training

    model = build_CNN(xo_train, xd_train, learning_rate, momentum, initializer_seed, mtype = 'regression')
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    model_r2.append(r2)
    model_MSE.append(MSE)
    model_pearson.append(pearson)


In [ ]:
table_make('CNN_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
           

# General Metrics - Early integration

In [12]:
print(f'X_main_phos shape: {X_main_phos.shape} y_main_phos shape: {y_main_phos.shape}')
print(f'X_main_onehot shape: {X_main_onehot.shape} y_main_onehot shape: {y_main_phos.shape}')

X_main_phos shape: (11583, 2401) y_main_phos shape: (11583,)
X_main_onehot shape: (11583, 383) y_main_onehot shape: (11583,)


In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_MSE = []
model_pearson = []

for seed in rs:
    
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    
    
    
    print(f'Mixed-Set Model')
    #MIXED SET
    ## Code for model building and training
    
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_onehot, y=y_main_phos, test_size=0.2, random_state = seed)

    model = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict(X_test)  
    r2 = r2_score(y_test, y_pred)
        
    mixed_set_r2.append(r2)

    
    
    
    print(f'Model: CNN')
    
    ## Code for model building and training
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    
    model = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict(X_test)  
    r2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    model_r2.append(r2)
    model_MSE.append(MSE)
    model_pearson.append(pearson)


In [ ]:
table_make('CNN_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/Nikhil code/NC_DeepIC50_metrics.csv')
           

# Plots

In [ ]:
dtype = 'phospho'

metrics1 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics2 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')
metrics3 = pd.read_csv(f'plots/phospho/Nikhil code/NC_DeepIC50_metrics.csv')
metrics4 = pd.read_csv(f'plots/phospho/Nikhil code/NC_EarlyDNN_metrics.csv')
metrics5 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics.csv')
metrics6 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics.csv')
metrics7 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics_Top2056feats.csv')
metrics8 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics_Top2056feats.csv')
metrics9 = pd.read_csv(f'plots/{dtype}/NormalisedX_rfr_metrics.csv')
metrics10 = pd.read_csv(f'plots/{dtype}/NormalisedX_xgb_metrics.csv')
metrics11 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(4)_metrics_L1000.csv')
metrics12 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_CNN_metrics_L1000.csv')

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))

# Plot 1
axes[0].plot(metrics1['rs'].astype(str), metrics1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0].plot(metrics1['rs'].astype(str), metrics1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0].set_title('CNN')
axes[0].set_ylabel('r-squared')

# Plot 2
axes[1].plot(metrics2['rs'].astype(str), metrics2['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1].plot(metrics2['rs'].astype(str), metrics2['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1].set_title('DNN')

# Plot 3
axes[2].plot(metrics3['rs'].astype(str), metrics3['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[2].plot(metrics3['rs'].astype(str), metrics3['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[2].plot(metrics3['rs'].astype(str), metrics3['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[2].set_title('DeepIC50')

# Plot 4
axes[3].plot(metrics4['rs'].astype(str), metrics4['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[3].plot(metrics4['rs'].astype(str), metrics4['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[3].plot(metrics4['rs'].astype(str), metrics4['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[3].set_title('Early integration DNN')
axes[3].legend(["mean model", "mixed-set model", "NN model"], loc=0)

axes[0].set_ylim(0, 0.85)
axes[1].set_ylim(0, 0.85)
axes[2].set_ylim(0, 0.85)
axes[3].set_ylim(0, 0.85)


fig.savefig('plots/all_nn_models_line.png')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Plot 1
axes[0,0].plot(metrics2['rs'].astype(str), metrics1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,0].plot(metrics2['rs'].astype(str), metrics1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,0].plot(metrics2['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,0].set_title('Late Integration DNN_ls')
axes[0,0].set_xlabel('seed')
axes[0,0].set_ylabel('R-squared')

# Plot 2
axes[1,0].plot(metrics11['rs'].astype(str), metrics11['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,0].plot(metrics11['rs'].astype(str), metrics11['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,0].plot(metrics11['rs'].astype(str), metrics11['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,0].set_title('Late Integration DNN_norm')
axes[1,0].set_xlabel('seed')

# Plot 3
axes[0,1].plot(metrics1['rs'].astype(str), metrics1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,1].plot(metrics1['rs'].astype(str), metrics1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,1].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,1].set_title('CNN_ls')
axes[0,1].set_xlabel('seed')

# Plot 4
axes[1,1].plot(metrics12['rs'].astype(str), metrics12['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,1].plot(metrics12['rs'].astype(str), metrics12['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,1].plot(metrics12['rs'].astype(str), metrics12['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,1].set_title('CNN_norm')
axes[1,1].set_xlabel('seed')


#fig.savefig('Targets/full_targets_comparison.png')

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))

# Plot 1
axes[0].plot(metrics5['rs'].astype(str), metrics5['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0].plot(metrics5['rs'].astype(str), metrics5['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(metrics5['rs'].astype(str), metrics5['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0].set_title('Random Forests-L1000')
axes[0].set_ylabel('r-squared')

# Plot 2
axes[1].plot(metrics6['rs'].astype(str), metrics6['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1].plot(metrics6['rs'].astype(str), metrics6['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1].plot(metrics6['rs'].astype(str), metrics6['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1].set_title('XGBoost-L1000')



# Plot 1
axes[2].plot(metrics7['rs'].astype(str), metrics7['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[2].plot(metrics7['rs'].astype(str), metrics7['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[2].plot(metrics7['rs'].astype(str), metrics7['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[2].set_title('Random Forests-feature_importances_')

# Plot 2
axes[3].plot(metrics8['rs'].astype(str), metrics8['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[3].plot(metrics8['rs'].astype(str), metrics8['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[3].plot(metrics8['rs'].astype(str), metrics8['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[3].set_title('XGBoost-feature_importances_')
axes[3].legend(["mean model", "mixed-set model", "model"], loc=0)

axes[0].set_ylim(0.50, 0.85)
axes[1].set_ylim(0.50, 0.85)
axes[2].set_ylim(0.50, 0.85)
axes[3].set_ylim(0.50, 0.85)

axes[1].yaxis.set_ticks([])
axes[2].yaxis.set_ticks([])
axes[3].yaxis.set_ticks([])

fig.savefig('plots/L1000_vs_feat_import.png')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1
axes[0].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(metrics5['rs'].astype(str), metrics5['model_r2'], linestyle='-', marker='.', color='green', linewidth=1)
axes[0].plot(metrics6['rs'].astype(str), metrics6['model_r2'], linestyle='-', marker='.', color='red', linewidth=1)
axes[0].set_title('Log scaled')
axes[0].set_ylabel('r-squared')
axes[0].set_ylim(0.50, 0.85)


# Plot 1

axes[1].plot(metrics12['rs'].astype(str), metrics12['model_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1].plot(metrics11['rs'].astype(str), metrics11['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1].plot(metrics9['rs'].astype(str), metrics9['model_r2'], linestyle='-', marker='.', color='green', linewidth=1)
axes[1].plot(metrics10['rs'].astype(str), metrics10['model_r2'], linestyle='-', marker='.', color='red', linewidth=1)


axes[1].set_title('Min-max scaled')
axes[1].legend(["CNN", "Late DNN", "Random Forest", "XGBoost"], loc=3)
axes[1].set_ylim(0.50, 0.85)

axes[1].yaxis.set_ticks([])

plt.tight_layout()

fig.savefig('plots/ls_vs_mm.png')

# Display the plots
plt.show()

## proteomic vs phospho

In [ ]:
prot1 = pd.read_csv('plots/proteomic/Nikhil code/NC_CNN_metrics.csv')
prot2 = pd.read_csv('plots/proteomic/Nikhil code/NC_Deep_NN(4)_metrics.csv')
prot3 = pd.read_csv('plots/proteomic/NormalisedX_rfr_metrics.csv')
prot4 = pd.read_csv('plots/proteomic/NormalisedX_xgb_metrics.csv')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1
axes[0,0].plot(metrics1['rs'].astype(str), metrics1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,0].plot(metrics1['rs'].astype(str), metrics1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,0].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,0].set_title('CNN-phosphoproteomic')
axes[0,0].set_ylabel('r-squared')

# Plot 2
axes[0,1].plot(metrics2['rs'].astype(str), metrics2['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,1].plot(metrics2['rs'].astype(str), metrics2['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,1].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,1].set_title('DNN-phosphoproteomic')

# Plot 1
axes[1,0].plot(prot1['rs'].astype(str), prot1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,0].plot(prot1['rs'].astype(str), prot1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,0].plot(prot1['rs'].astype(str), prot1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,0].set_title('CNN-proteomic')
axes[1,0].set_ylabel('r-squared')

# Plot 2
axes[1,1].plot(prot2['rs'].astype(str), prot2['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,1].plot(prot2['rs'].astype(str), prot2['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,1].plot(prot2['rs'].astype(str), prot2['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,1].set_title('DNN-proteomic')
axes[1,1].legend(["mean model", "mixed-set model", "model"], loc=0)

axes[0,0].set_ylim(0.5, 0.85)
axes[0,1].set_ylim(0.5, 0.85)
axes[1,0].set_ylim(0.5, 0.85)
axes[1,1].set_ylim(0.5, 0.85)

axes[0,1].yaxis.set_ticks([])
axes[1,1].yaxis.set_ticks([])

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()
fig.savefig('plots/NN_models_phospho_vs_proteomic.png')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1
axes[0,0].plot(metrics9['rs'].astype(str), metrics9['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,0].plot(metrics9['rs'].astype(str), metrics9['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,0].plot(metrics9['rs'].astype(str), metrics9['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,0].set_title('Random Forest-phospho')
axes[0,0].set_ylabel('R-squared')
axes[0,0].set_xlabel('seed')

# Plot 2
axes[0,1].plot(metrics10['rs'].astype(str), metrics10['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0,1].plot(metrics10['rs'].astype(str), metrics10['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0,1].plot(metrics10['rs'].astype(str), metrics10['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0,1].set_title('XGBoost-phospho')
axes[0,1].set_xlabel('seed')

# Plot 1
axes[1,0].plot(prot3['rs'].astype(str), prot3['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,0].plot(prot3['rs'].astype(str), prot3['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,0].plot(prot3['rs'].astype(str), prot3['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,0].set_title('Random Forest-proteomic')
axes[1,0].set_ylabel('R-squared')
axes[1,0].set_xlabel('seed')

# Plot 2
axes[1,1].plot(prot4['rs'].astype(str), prot4['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1,1].plot(prot4['rs'].astype(str), prot4['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1,1].plot(prot4['rs'].astype(str), prot4['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1,1].set_title('XGBoost-proteomic')
axes[1,1].set_xlabel('seed')
axes[1,1].legend(["mean model", "mixed-set model", "model"], loc=0)


# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()
fig.savefig('plots/DecisionTree_models_phospho_vs_proteomic.png')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1
axes[0].plot(prot1['rs'].astype(str), prot1['model_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0].plot(prot2['rs'].astype(str), prot2['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(prot3['rs'].astype(str), prot3['model_r2'], linestyle='-', marker='.', color='green', linewidth=1)
axes[0].plot(prot4['rs'].astype(str), prot4['model_r2'], linestyle='-', marker='.', color='red', linewidth=1)
axes[0].set_title('Proteomic')
axes[0].set_ylabel('r-squared')
axes[0].set_ylim(0.5, 0.85)

# Plot 1
axes[1].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1].plot(metrics9['rs'].astype(str), metrics9['model_r2'], linestyle='-', marker='.', color='green', linewidth=1)
axes[1].plot(metrics10['rs'].astype(str), metrics10['model_r2'], linestyle='-', marker='.', color='red', linewidth=1)
axes[1].set_title('Phosphproteomic')
axes[1].legend(["CNN", "Late DNN", "Random Forest", "XGBoost"], loc=3)
axes[1].set_ylim(0.5, 0.85)


axes[1].yaxis.set_ticks([])

plt.tight_layout()

fig.savefig('plots/Phospho_vs_proteomic.png')

# Display the plots
plt.show()


## DNN comparison

In [ ]:
level1 = pd.read_csv('plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(1)_metrics_L1000.csv')
level2 = pd.read_csv('plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(2)_metrics_L1000.csv')
level3 = pd.read_csv('plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(3)_metrics_L1000.csv')
level4 = pd.read_csv('plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(4)_metrics_L1000.csv')
level5 = pd.read_csv('plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(5)_metrics_L1000.csv')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 10))

# Plot 1
axes.plot(level1['rs'].astype(str), level1['model_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes.plot(level2['rs'].astype(str), level2['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes.plot(level3['rs'].astype(str), level3['model_r2'], linestyle='-', marker='.', color='green', linewidth=1)
axes.plot(level4['rs'].astype(str), level4['model_r2'], linestyle='-', marker='.', color='red', linewidth=1)
axes.plot(level5['rs'].astype(str), level5['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes.set_title('Deep_NN model comparison-levels of complexity')
axes.set_xlabel('seed')
axes.set_ylabel('R-squared')

axes.legend(["level1", "level2", "level3", "level4", "level5"], loc=0)

plt.tight_layout()

# Display the plots
plt.show()

fig.savefig('plots/phospho/NormalisedX_L1000/Deep_NN_model_comparison.png')

## bar charts

In [ ]:
metrics5 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics.csv')
metrics7 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics_Top2056feats.csv')
metrics9 = pd.read_csv(f'plots/phospho/NormalisedX_rfr_metrics.csv')
rfr_metrics = [mean(metrics5['model_r2']), mean(metrics7['model_r2']), mean(metrics9['model_r2'])]
rfr_metrics_sd = [np.std(metrics5['model_r2']), np.std(metrics7['model_r2']), np.std(metrics9['model_r2'])]

metrics6 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics.csv')
metrics8 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics_Top2056feats.csv')
metrics10 = pd.read_csv(f'plots/phospho/NormalisedX_xgb_metrics.csv')
xgb_metrics = [mean(metrics6['model_r2']), mean(metrics8['model_r2']), mean(metrics10['model_r2'])]
xgb_metrics_sd = [np.std(metrics6['model_r2']), np.std(metrics8['model_r2']), np.std(metrics10['model_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("L1000_ls", "feature_importances", "L1000_norm")
r_squared_means = {
    'rfr': rfr_metrics,
    'xgb': xgb_metrics
}

r_squared_std = {
    'rfr_sd': rfr_metrics_sd,
    'xgb_sd': xgb_metrics_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.20  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['green', 'red']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('feature selection comparisons for Decision Trees')
ax.set_xticks(x + width * (multiplier - 1))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='upper left')
ax.set_ylim(0.5, 0.9)

fig.savefig('plots/phospho/feat_select_methods_DecisionTree.png')

In [ ]:
metrics1 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics12 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_CNN_metrics_L1000.csv')
CNN_metrics = [mean(metrics1['model_r2']), mean(metrics12['model_r2'])]
CNN_metrics_sd = [np.std(metrics1['model_r2']), np.std(metrics12['model_r2'])]

metrics2 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')
metrics11 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(4)_metrics_L1000.csv')
DNN_metrics = [mean(metrics2['model_r2']), mean(metrics11['model_r2'])]
DNN_metrics_sd = [np.std(metrics2['model_r2']), np.std(metrics11['model_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("L1000_ls", "L1000_norm")
r_squared_means = {
    'CNN': CNN_metrics,
    'DNN': DNN_metrics
}

r_squared_std = {
    'CNN_sd': CNN_metrics_sd,
    'DNN_sd': DNN_metrics_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.20  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    ax.bar_label(rects, padding=27)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('feature selection comparisons for Neural Networks')
ax.set_xticks(x + width * (multiplier - 1))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='upper left')
ax.set_ylim(0.5, 0.9)

fig.savefig('plots/phospho/feat_select_methods_Neural_Networks.png')

In [ ]:
CNN_prot = pd.read_csv('plots/proteomic/Nikhil code/NC_CNN_metrics.csv')
CNN_phos = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
CNN = [mean(CNN_prot['model_r2']), mean(CNN_phos['model_r2'])]
CNN_sd = [np.std(CNN_prot['model_r2']), np.std(CNN_phos['model_r2'])]

DNN_prot = pd.read_csv('plots/proteomic/Nikhil code/NC_Deep_NN(4)_metrics.csv')
DNN_phos = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')
DNN = [mean(DNN_prot['model_r2']), mean(DNN_phos['model_r2'])]
DNN_sd = [np.std(DNN_prot['model_r2']), np.std(DNN_phos['model_r2'])]

rfr_prot = pd.read_csv('plots/proteomic/NormalisedX_rfr_metrics.csv')
rfr_phos = pd.read_csv(f'plots/phospho/NormalisedX_rfr_metrics.csv')
rfr = [mean(rfr_prot['model_r2']), mean(rfr_phos['model_r2'])]
rfr_sd = [np.std(rfr_prot['model_r2']), np.std(rfr_phos['model_r2'])]


xgb_prot = pd.read_csv('plots/proteomic/NormalisedX_xgb_metrics.csv')
xgb_phos = pd.read_csv(f'plots/phospho/NormalisedX_xgb_metrics.csv')
xgb = [mean(xgb_prot['model_r2']), mean(xgb_phos['model_r2'])]
xgb_sd = [np.std(xgb_prot['model_r2']), np.std(xgb_phos['model_r2'])]


In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("proteomic", "phospho")
r_squared_means = {
    'CNN': CNN,
    'DNN': DNN,
    'rfr': rfr,
    'xgb': xgb
}

# Calculate standard deviations for each model
r_squared_std = {
    'CNN_sd': CNN_sd,
    'DNN_sd': DNN_sd,
    'rfr_sd': rfr_sd,
    'xgb_sd': xgb_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.15  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['orange', 'yellow', 'green', 'red']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    ax.bar_label(rects, padding=25)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('Proteomic vs Phospho data')
ax.set_xticks(x + width * (multiplier - 1))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='lower left')
ax.set_ylim(0.5, 0.9)

fig.savefig('plots/phospho_vs_proteomic.png')

In [ ]:
metrics1 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics12 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_CNN_metrics_L1000.csv')
CNN = [mean(metrics1['model_r2']), mean(metrics12['model_r2'])]
CNN_sd = [np.std(metrics1['model_r2']), np.std(metrics12['model_r2'])]

metrics2 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')
metrics11 = pd.read_csv(f'plots/phospho/NormalisedX_L1000/NormalisedX_LateDNN(4)_metrics_L1000.csv')
DNN = [mean(metrics2['model_r2']), mean(metrics11['model_r2'])]
DNN_sd = [np.std(metrics2['model_r2']), np.std(metrics11['model_r2'])]

metrics5 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics.csv')
metrics9 = pd.read_csv(f'plots/phospho/NormalisedX_rfr_metrics.csv')
rfr = [mean(metrics5['model_r2']), mean(metrics9['model_r2'])]
rfr_sd = [np.std(metrics5['model_r2']), np.std(metrics9['model_r2'])]

metrics6 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics.csv')
metrics10 = pd.read_csv(f'plots/phospho/NormalisedX_xgb_metrics.csv')
xgb = [mean(metrics6['model_r2']), mean(metrics10['model_r2'])]
xgb_sd = [np.std(metrics6['model_r2']), np.std(metrics10['model_r2'])]


In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("log scaled", "Min-Max scaled")
r_squared_means = {
    'CNN': CNN,
    'DNN': DNN,
    'rfr': rfr,
    'xgb': xgb
}

# Calculate standard deviations for each model
r_squared_std = {
    'CNN_sd': CNN_sd,
    'DNN_sd': DNN_sd,
    'rfr_sd': rfr_sd,
    'xgb_sd': xgb_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['orange', 'yellow', 'green', 'red']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('Log-scaled vs Min Max scaled')
ax.set_xticks(x + width * (multiplier - 1))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='lower left')
ax.set_ylim(0, 0.9)

fig.savefig('plots/phospho/ls_vs_norm.png')

In [ ]:
metrics1 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics2 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')
metrics3 = pd.read_csv(f'plots/phospho/Nikhil code/NC_DeepIC50_metrics.csv')
metrics4 = pd.read_csv(f'plots/phospho/Nikhil code/NC_EarlyDNN_metrics.csv')

model_r2 = [mean(metrics1['model_r2']), mean(metrics2['model_r2']), mean(metrics3['model_r2']), mean(metrics4['model_r2'])]
model_r2_sd = [np.std(metrics1['model_r2']), np.std(metrics2['model_r2']), np.std(metrics3['model_r2']), np.std(metrics4['model_r2'])]

mm_r2 = [mean(metrics1['mm_r2']), mean(metrics2['mm_r2']), mean(metrics3['mm_r2']), mean(metrics4['mm_r2'])]
mm_r2_sd = [np.std(metrics1['mm_r2']), np.std(metrics2['mm_r2']), np.std(metrics3['mm_r2']), np.std(metrics4['mm_r2'])]

mixed_r2 = [mean(metrics1['mixed_set_r2']), mean(metrics2['mixed_set_r2']), mean(metrics3['mixed_set_r2']), mean(metrics4['mixed_set_r2'])]
mixed_r2_sd = [np.std(metrics1['mixed_set_r2']), np.std(metrics2['mixed_set_r2']), np.std(metrics3['mixed_set_r2']), np.std(metrics4['mixed_set_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("CNN", "DNN", "DeepIC50", "Early integration DNN")
r_squared_means = {
    'model': model_r2,
    'mean model': mm_r2,
    'mixed-set model': mixed_r2
}

# Calculate standard deviations for each model
r_squared_std = {
    'model_sd': model_r2_sd,
    'mean model_sd': mm_r2_sd,
    'mixed-set model_sd': mixed_r2_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['blue', 'orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    #ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
#ax.set_title('Neural Network Comparison')
ax.set_xticks(x + width * (multiplier - 2))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='upper right')
ax.set_ylim(0, 1)

fig.savefig('plots/phospho/nn_performance.png')

In [ ]:
metrics1 = pd.read_csv('plots/proteomic/Nikhil code/NC_rfr_metrics.csv')
metrics2 = pd.read_csv('plots/proteomic/Nikhil code/NC_xgb_metrics.csv')
metrics3 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics.csv')
metrics4 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics.csv')

model_r2 = [mean(metrics1['model_r2']), mean(metrics2['model_r2']), mean(metrics3['model_r2']), mean(metrics4['model_r2'])]
model_r2_sd = [np.std(metrics1['model_r2']), np.std(metrics2['model_r2']), np.std(metrics3['model_r2']), np.std(metrics4['model_r2'])]

mm_r2 = [mean(metrics1['mm_r2']), mean(metrics2['mm_r2']), mean(metrics3['mm_r2']), mean(metrics4['mm_r2'])]
mm_r2_sd = [np.std(metrics1['mm_r2']), np.std(metrics2['mm_r2']), np.std(metrics3['mm_r2']), np.std(metrics4['mm_r2'])]

mixed_r2 = [mean(metrics1['mixed_set_r2']), mean(metrics2['mixed_set_r2']), mean(metrics3['mixed_set_r2']), mean(metrics4['mixed_set_r2'])]
mixed_r2_sd = [np.std(metrics1['mixed_set_r2']), np.std(metrics2['mixed_set_r2']), np.std(metrics3['mixed_set_r2']), np.std(metrics4['mixed_set_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("Random Forest Proteomic", "XGBoost Proteomic", "Random Forest Phosphoproteomic", "XGBoost Phosphoproteomic")
r_squared_means = {
    'model': model_r2,
    'mean model': mm_r2,
    'mixed-set model': mixed_r2
}

# Calculate standard deviations for each model
r_squared_std = {
    'model_sd': model_r2_sd,
    'mean model_sd': mm_r2_sd,
    'mixed-set model_sd': mixed_r2_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['blue', 'orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    #ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('Decision trees')
ax.set_xticks(x + width * (multiplier - 2))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='upper right')
ax.set_ylim(0.5, 0.8)

fig.savefig('plots/DecisionTree_phospho_vs_proteomic_barChart.png')

In [ ]:
metrics1 = pd.read_csv('plots/proteomic/Nikhil code/NC_CNN_metrics.csv')
metrics2 = pd.read_csv('plots/proteomic/Nikhil code/NC_Deep_NN(4)_metrics.csv')
metrics3 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics4 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')

model_r2 = [mean(metrics1['model_r2']), mean(metrics2['model_r2']), mean(metrics3['model_r2']), mean(metrics4['model_r2'])]
model_r2_sd = [np.std(metrics1['model_r2']), np.std(metrics2['model_r2']), np.std(metrics3['model_r2']), np.std(metrics4['model_r2'])]

mm_r2 = [mean(metrics1['mm_r2']), mean(metrics2['mm_r2']), mean(metrics3['mm_r2']), mean(metrics4['mm_r2'])]
mm_r2_sd = [np.std(metrics1['mm_r2']), np.std(metrics2['mm_r2']), np.std(metrics3['mm_r2']), np.std(metrics4['mm_r2'])]

mixed_r2 = [mean(metrics1['mixed_set_r2']), mean(metrics2['mixed_set_r2']), mean(metrics3['mixed_set_r2']), mean(metrics4['mixed_set_r2'])]
mixed_r2_sd = [np.std(metrics1['mixed_set_r2']), np.std(metrics2['mixed_set_r2']), np.std(metrics3['mixed_set_r2']), np.std(metrics4['mixed_set_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("CNN Proteomic", "DNN Proteomic", "CNN Phosphoproteomic", "DNN Phosphoproteomic")
r_squared_means = {
    'model': model_r2,
    'mean model': mm_r2,
    'mixed-set model': mixed_r2
}

# Calculate standard deviations for each model
r_squared_std = {
    'model_sd': model_r2_sd,
    'mean model_sd': mm_r2_sd,
    'mixed-set model_sd': mixed_r2_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['blue', 'orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    #ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('Neural networks')
ax.set_xticks(x + width * (multiplier - 2))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='lower left')
ax.set_ylim(0.5, 0.8)

fig.savefig('plots/NN_phospho_vs_proteomic_barChart.png')

In [ ]:
metrics1 = pd.read_csv(f'plots/phospho/Nikhil code/NC_xgb_metrics.csv')
metrics2 = pd.read_csv(f'plots/phospho/Nikhil code/NC_rfr_metrics.csv')
metrics3 = pd.read_csv(f'plots/phospho/Nikhil code/NC_CNN_metrics.csv')
metrics4 = pd.read_csv(f'plots/phospho/Nikhil code/NC_LateDNN(4)_metrics.csv')

model_r2 = [mean(metrics1['model_r2']), mean(metrics2['model_r2']), mean(metrics3['model_r2']), mean(metrics4['model_r2'])]
model_r2_sd = [np.std(metrics1['model_r2']), np.std(metrics2['model_r2']), np.std(metrics3['model_r2']), np.std(metrics4['model_r2'])]

mm_r2 = [mean(metrics1['mm_r2']), mean(metrics2['mm_r2']), mean(metrics3['mm_r2']), mean(metrics4['mm_r2'])]
mm_r2_sd = [np.std(metrics1['mm_r2']), np.std(metrics2['mm_r2']), np.std(metrics3['mm_r2']), np.std(metrics4['mm_r2'])]

mixed_r2 = [mean(metrics1['mixed_set_r2']), mean(metrics2['mixed_set_r2']), mean(metrics3['mixed_set_r2']), mean(metrics4['mixed_set_r2'])]
mixed_r2_sd = [np.std(metrics1['mixed_set_r2']), np.std(metrics2['mixed_set_r2']), np.std(metrics3['mixed_set_r2']), np.std(metrics4['mixed_set_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("XGBoost", "Random Forest", "CNN", "DNN")
r_squared_means = {
    'model': model_r2,
    'mean model': mm_r2,
    'mixed-set model': mixed_r2
}

# Calculate standard deviations for each model
r_squared_std = {
    'model_sd': model_r2_sd,
    'mean model_sd': mm_r2_sd,
    'mixed-set model_sd': mixed_r2_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['blue', 'orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    #ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
#ax.set_title('All models Phospho Comparison')
ax.set_xticks(x + width * (multiplier - 2))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='upper left')
ax.set_ylim(0.5, 0.8)

fig.savefig('plots/phospho/All_models_phospho_barChart.png')

In [ ]:
metrics1 = pd.read_csv(f'plots/proteomic/NormalisedX_xgb_metrics.csv')
metrics2 = pd.read_csv(f'plots/proteomic/NormalisedX_rfr_metrics.csv')
metrics3 = pd.read_csv(f'plots/proteomic/Nikhil code/NC_CNN_metrics.csv')
metrics4 = pd.read_csv(f'plots/proteomic/Nikhil code/NC_Deep_NN(4)_metrics.csv')

model_r2 = [mean(metrics1['model_r2']), mean(metrics2['model_r2']), mean(metrics3['model_r2']), mean(metrics4['model_r2'])]
model_r2_sd = [np.std(metrics1['model_r2']), np.std(metrics2['model_r2']), np.std(metrics3['model_r2']), np.std(metrics4['model_r2'])]

mm_r2 = [mean(metrics1['mm_r2']), mean(metrics2['mm_r2']), mean(metrics3['mm_r2']), mean(metrics4['mm_r2'])]
mm_r2_sd = [np.std(metrics1['mm_r2']), np.std(metrics2['mm_r2']), np.std(metrics3['mm_r2']), np.std(metrics4['mm_r2'])]

mixed_r2 = [mean(metrics1['mixed_set_r2']), mean(metrics2['mixed_set_r2']), mean(metrics3['mixed_set_r2']), mean(metrics4['mixed_set_r2'])]
mixed_r2_sd = [np.std(metrics1['mixed_set_r2']), np.std(metrics2['mixed_set_r2']), np.std(metrics3['mixed_set_r2']), np.std(metrics4['mixed_set_r2'])]

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

metrics = ("XGBoost Proteomic", "Random Forest Proteomic", "CNN Proteomic", "Deep_NN Proteomic")
r_squared_means = {
    'model': model_r2,
    'mean model': mm_r2,
    'mixed-set model': mixed_r2
}

# Calculate standard deviations for each model
r_squared_std = {
    'model_sd': model_r2_sd,
    'mean model_sd': mm_r2_sd,
    'mixed-set model_sd': mixed_r2_sd
}

x = np.arange(len(metrics))  # the label locations
width = 0.10  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(constrained_layout=True)

# Define custom colors for each model's bars
colors = ['blue', 'orange', 'yellow']

for model, r in r_squared_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, r, width, label=model, color=colors[multiplier])
    #ax.bar_label(rects, padding=10)
    ax.errorbar(x + offset, r, yerr=r_squared_std[model+'_sd'], fmt='none', color='black', capsize=5)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('r-squared')
ax.set_title('All models Proteomic Comparison')
ax.set_xticks(x + width * (multiplier - 1))  # Adjusted the tick positions
ax.set_xticklabels(metrics)
ax.legend(loc='lower left')
ax.set_ylim(0.5, 0.8)

fig.savefig('plots/phospho/All_models_proteomic_barChart.png')